In [1]:
import torch
device= 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [2]:
with open(r"C:\Users\amogh\Downloads\wizard_of_oz.txt", 'r', encoding= 'utf-8') as f:
    text= f.read()

chars= sorted(set(text))
vocab_size= len(chars)
print(chars)

          

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']


Now we create a tokeniser, to convert each element of the chars array to an integer.

In [3]:
string_to_int= {ch:i for i, ch in enumerate(chars) }
int_to_string= {i:ch for i, ch in enumerate(chars)}
encode= lambda s: [string_to_int[c] for c in s]
decode= lambda l: ''.join(int_to_string[i] for i in l)
#This is a very simple mapping for a small vocabulary


A bigram language model is called so, because it predicts a character based on the previous character.

Example:
Input string= hello

start->h

h->e

e->l

l->l

l->o

The prediction shall be made on the basis of the probability of succeeding characters, as determined by studying the corpus
. 

In [4]:
data= torch.tensor(encode(text), dtype= torch.long)

In [5]:
data[:100]

tensor([80,  1,  1, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32, 29,
         1, 47, 33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0,  1,  1, 26,
        49,  0,  0,  1,  1, 36, 11,  1, 30, 42, 25, 38, 35,  1, 26, 25, 45, 37,
         0,  0,  1,  1, 25, 45, 44, 32, 39, 42,  1, 39, 30,  1, 44, 32, 29,  1,
        47, 33, 50, 25, 42, 28,  1, 39, 30,  1, 39, 50,  9,  1, 44, 32, 29,  1,
        36, 25, 38, 28,  1, 39, 30,  1, 39, 50])

In [6]:
n= int(0.71*len(data))
train_data= data[:n]
val_data= data[n:]



block_size= 8
batch_size= 4

def get_batch(split):
    if split== 'train':
        data= train_data
    else: data= val_data
    ix= torch.randint(len(data)-block_size, (batch_size, ))
    print(ix)
    x= torch.stack([data[i:i+block_size] for i in ix])
    y= torch.stack([data[i+1:i+block_size+1] for i in ix])
    x,y= x.to(device), y.to(device)
    return x,y
    
x,y= get_batch('train')

print('inputs')
print(x)
print('targets')
print(y)

tensor([ 84652, 141886, 133872,  41450])
inputs
tensor([[ 1, 72, 69, 65, 58, 67, 57, 62],
        [67, 57, 23,  1, 76, 58,  1, 76],
        [11,  3,  0,  0, 44, 61, 58,  1],
        [ 1, 76, 54, 73, 56, 61, 58, 57]], device='cuda:0')
targets
tensor([[72, 69, 65, 58, 67, 57, 62, 57],
        [57, 23,  1, 76, 58,  1, 76, 62],
        [ 3,  0,  0, 44, 61, 58,  1, 72],
        [76, 54, 73, 56, 61, 58, 57,  1]], device='cuda:0')


## Word Embedding

Embeddings are a means to convert words into tokens. These tokens are in the form of vectors/tensors. What is essential here is the knowledge of the vocabulary size, and a determination of the dimensions of the resultant vectors. 
An embedding layer is created using these as inputs; it is then used to look up/identify the embeddings for a given set of indices.

In [7]:
import torch.nn as nn
from torch.nn import functional as F



In [9]:
class BigramLanguageModel(nn.Module):
    #Define the Embedding table
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table= nn.Embedding(vocab_size, vocab_size)
    
    #Define the forward pass for the network
    def forward(self, index, targets= None):
        logits=self.token_embedding_table(index)
        #logits are of the form floating point numbers that have been normalised
        
        if targets is None:
            loss= None
        else:
            B, T, C= logits.shape
        #B is for Batches, T is for Targets, C is for 
            logits= logits.view(B*T,C)
            targets= targets.view(B*T)
            loss= F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, index, max_new_tokens):
        #index is (B,T) array of indices in current context (context as in context of tokens in the bigram) 
        for _ in range (max_new_tokens):
            logits,loss= self.forward(index) #gets predictions
            logits= logits[:,-1,:] #focussing on the last time step- property of the bigram, becomes (B,C)  
            probs= F.softmax(logits, dim= -1) # (B,C)
            
            index_next= torch.multinomial(probs, num_samples= 1)
            index= torch.cat((index, index_next), dim= 1) #for the new context
            
            return index
    
model= BigramLanguageModel(vocab_size)
m= model.to(device) 

context= torch.zeros((1,1), dtype= torch.long, device= device)
generated_chars= decode(m.generate(context, max_new_tokens= 500)[0].tolist())
print(generated_chars)


u


In [10]:
learning_rate= 4e-3
max_iters= 2000
optimiser= torch.optim.AdamW(model.parameters(), lr= learning_rate)

for iter in range(max_iters):
    
    xb, yb= get_batch('train')
    
    logits, loss= model.forward(xb, yb)
    optimiser.zero_grad(set_to_none= True)
    loss.backward()
    optimiser.step()
    
print(loss.item())
    

tensor([82187, 71844, 41144, 59979])
tensor([144778, 145625,  62812,  87832])
tensor([157798,  51877,  34135,  41527])
tensor([159753,  86237, 137522, 153757])
tensor([122213,  27762,  60657,  63538])
tensor([ 11982,  71612, 109037,  85797])
tensor([ 57095,  20850, 127442,  59530])
tensor([33417,  2412, 70043, 19100])
tensor([11958, 49188, 84214, 80465])
tensor([151922,  53187, 135742, 123676])
tensor([127081, 101331,  66838, 162339])
tensor([ 24454,   2666, 124781, 163175])
tensor([ 70134,  54448, 100784,  96322])
tensor([109052,  62976,  36360, 137511])
tensor([104671,  70708,  99698,  57759])
tensor([78228, 84964, 14374, 81841])
tensor([ 4653, 54683, 22357, 84565])
tensor([ 12036, 146858, 101509, 160893])
tensor([ 59513, 153795, 117720,  24833])
tensor([136765,  11954, 110240,  24293])
tensor([20196, 97181, 91872, 30786])
tensor([132530, 140769,   4656, 158324])
tensor([ 49130,  29967, 151399, 100959])
tensor([151105, 120401, 147924, 161761])
tensor([ 35612,  86047, 144884,  22752])

tensor([146059,  21603, 143300,  77177])
tensor([162851,  30343, 112095,  73548])
tensor([  1523, 128868, 163477,  38554])
tensor([ 67015,  14147, 149746,  62955])
tensor([  8304, 160337, 124540,   5018])
tensor([27712, 30544, 64647, 16232])
tensor([112422,  51219, 162354,  61644])
tensor([ 19696,   5798, 128638, 110945])
tensor([38933, 69323, 87701, 85530])
tensor([ 13406, 161945, 144584,  76022])
tensor([ 77461,  49705,  20861, 132487])
tensor([159595,  62562,  25856, 107042])
tensor([101300, 113290,  15224, 144777])
tensor([ 43235,  29476, 158208, 116023])
tensor([ 64414, 156078, 128100,  91070])
tensor([157235,  64196,    659,  59284])
tensor([ 86144, 132837, 160938,  36726])
tensor([122092,  23230, 149963, 143067])
tensor([113565,  89338,  58343,  82964])
tensor([ 16398,  26820, 145630, 130249])
tensor([161189,  71596, 141269, 142760])
tensor([ 31876,  83062,  40114, 146758])
tensor([115490,  36852,  75454,  72632])
tensor([ 64226, 140812,  34148, 148404])
tensor([120113, 136153, 

tensor([ 62738,  31122, 111945, 131249])
tensor([130383,  33883, 125747,  11906])
tensor([ 99824,  51521,  98469, 138575])
tensor([ 88466, 144053,  59619, 153039])
tensor([105939, 136842,  58897,  54782])
tensor([116018,  48120, 128180,  52138])
tensor([142678, 126308, 128946,   5829])
tensor([ 20292,  11151,  11149, 164643])
tensor([108933,  74372,   8704,   9519])
tensor([ 24852,  74799, 155361, 158869])
tensor([142270,  37970, 103237,  26656])
tensor([   485, 148392,  61088, 105036])
tensor([ 66999, 140243, 163636,  73725])
tensor([ 42433, 141041, 107374, 138296])
tensor([13281, 34648,  5663,  9736])
tensor([21031, 78035, 14021,  6994])
tensor([134466,  91071, 151059, 154279])
tensor([133900,  55217,   6636,  21647])
tensor([ 22995,  12976,  43167, 121015])
tensor([ 37088,  62586, 115769,  84369])
tensor([101603,  63891, 147756,  57009])
tensor([107189,  23851,  10717, 153688])
tensor([63630, 60472, 23196, 63917])
tensor([10166, 31976, 45966,  6910])
tensor([69577, 93903, 17512, 869

tensor([158618,  70854, 139108, 156032])
tensor([32956, 29660, 86164, 44070])
tensor([114359, 152676,  29183,  75072])
tensor([ 49814, 129923, 126869,  37334])
tensor([104322,  13618,  13013,  78607])
tensor([50263, 99000, 52229, 17689])
tensor([  1455,  96077, 134327,  52107])
tensor([117308, 162783,  31952,  81864])
tensor([ 99927, 115300,  15422, 153654])
tensor([158222,  77539,   1151,  81025])
tensor([ 13227, 155200, 123131, 118192])
tensor([ 95672, 115107,  91490,  47507])
tensor([143214,    536, 143280, 112529])
tensor([ 43162,  37205, 112354, 164425])
tensor([160238,  59069,  61151,  95188])
tensor([ 25830, 137020,  21001,  87906])
tensor([  6897,  80662, 157968, 143396])
tensor([  7036, 124822,  56412,  92570])
tensor([  5755,  87312,  54856, 149616])
tensor([152202,  15736,  15475,  18976])
tensor([ 53809, 106383, 113575,  14084])
tensor([ 61476,  47347, 114345, 145513])
tensor([32616, 44610, 30164, 63392])
tensor([ 95248, 163694, 141383,  90680])
tensor([140854,  33846,  304

tensor([134175, 152217, 139312,  75338])
tensor([151143,  42994,  81148,  77029])
tensor([ 36193,  22775, 139439,  54076])
tensor([157946, 113906,  55649,  70508])
tensor([ 80072,  75265, 125593,  72895])
tensor([128705, 125313, 162060, 111281])
tensor([ 74784, 133771,  33004,   4664])
tensor([ 16560, 141376,  36938,  33274])
tensor([88117, 39608, 23331, 59398])
tensor([116355,  10442,  46039,  59972])
tensor([ 76356, 113235,  66898, 143826])
tensor([111884, 143634,  30874,  84291])
tensor([150313, 140706, 132599, 110522])
tensor([119254,  98067,  59566,  22353])
tensor([135122, 132817,   8169,  63444])
tensor([ 30743, 110939, 158335,  19741])
tensor([80705, 69906, 32305, 46705])
tensor([ 80575,  65903, 158315,  98372])
tensor([ 58958, 150695,  94732, 152653])
tensor([ 62399, 121163,  60340, 146406])
tensor([95163, 97715, 92190, 34478])
tensor([127275,  21629,  38335, 120370])
tensor([161375, 110312,  65706, 151088])
tensor([130776,  83466, 120093, 128273])
tensor([ 17069, 151744,  882

tensor([150175,   8016, 117455,  38244])
tensor([164513, 122154,  96694,  29094])
tensor([98975, 34269, 83131,  3885])
tensor([119027, 135235,  84836,  33725])
tensor([ 15860, 124433, 135321,  12433])
tensor([150007,  96519, 128520,  79867])
tensor([ 89795,  11050, 150756, 118422])
tensor([107417, 158363,  53482,  46886])
tensor([81788, 82210, 96090, 60903])
tensor([103188, 131169, 102792,  11977])
tensor([115380, 102996, 158146,   3292])
tensor([119940,  42445, 131396, 147395])
tensor([ 28944, 115396, 159167,  13682])
tensor([ 88373,  36216,  27956, 110852])
tensor([ 50070,  83211, 122525, 161923])
tensor([160223,  88484,  30021,  46808])
tensor([44582, 25457, 49493, 97242])
tensor([ 93448,  65692, 109329,  26696])
tensor([61658,  6983, 18906, 91446])
tensor([138262, 100683,  50117,  56535])
tensor([120624, 141668,  42169,  90749])
tensor([146117, 107371, 155060, 107660])
tensor([41400, 84524, 35543, 65147])
tensor([103981, 160536, 143931,  11467])
tensor([146435, 118206,  17163, 1182

tensor([156577,  86589, 122727,  91985])
tensor([150008, 120210, 161544,  45942])
tensor([ 50174, 100600,  45684,   3745])
tensor([ 37163,  15916,  21177, 144353])
tensor([ 99010,  15719, 159244, 161467])
tensor([ 96041, 145141, 126422,  56074])
tensor([ 72089,  37340, 137479,   7750])
tensor([108520, 111806, 139051,  58586])
tensor([135636, 140438, 104045, 112942])
tensor([ 44407,  75187,  53032, 134362])
tensor([156637,  15058, 148306,  99438])
tensor([ 50029,  81764,  58198, 144580])
tensor([ 47175, 133570,  38982, 134333])
tensor([ 42059, 130823,  42426,  42348])
tensor([ 43796, 163321,  41608,  52184])
tensor([ 75406,   9919,  22778, 116153])
tensor([   913, 104157,  12650,  95160])
tensor([142899,  91975,  58056,  84210])
tensor([ 46559,  50202, 124304, 108213])
tensor([ 66128, 157615,  43033,  43193])
tensor([ 98981,  76234, 108875, 161493])
tensor([151452, 150400, 162005,  89197])
tensor([19247, 27175, 13200, 93210])
tensor([ 22421, 115310, 111130,  26942])
tensor([31231, 60085

tensor([147076, 134463, 105622,  22351])
tensor([ 91037,  95844, 121936,   7307])
tensor([137213,  97940,  56929, 101382])
tensor([  452, 48359, 93355, 19650])
tensor([110997,   3442, 158371,   6361])
tensor([143770,  96602,  55629, 144120])
tensor([149834,  90688, 153928,  46262])
tensor([ 18833,   1865,  62428, 145351])
tensor([134751, 121878,  45212, 140850])
tensor([ 73335,  89489, 132521,  68741])
tensor([57454,  5800, 45538, 94459])
tensor([137271, 126114, 156488, 129521])
tensor([155980, 109623,  17871,  58260])
tensor([ 9738, 56968, 61971, 66374])
tensor([78908, 10837, 92393, 53725])
tensor([101169,  66864,  35404, 147333])
tensor([119837,  34865, 161469,   5116])
tensor([125431,  41893,   1238,  28970])
tensor([111689, 123393,   9413,  44478])
tensor([ 96209, 163390, 108418, 103844])
tensor([135167,  87640,  88245,  10676])
tensor([ 78582, 164443,  45337,  41666])
tensor([  5085,   5250, 106449, 148187])
tensor([23379, 78064, 99054, 46103])
tensor([109131,  55668,  45226, 1008

tensor([ 22153, 112877, 136447,  91144])
tensor([155071,  80374,  39087,  86841])
tensor([158220, 157364, 101809, 128404])
tensor([ 85045, 132270, 140153,  30727])
tensor([156752, 139105,  57055,  88115])
tensor([ 52519, 124424,  19286, 108494])
tensor([ 90183,  50104, 128519, 161776])
tensor([109614,  26924, 145760, 159306])
tensor([ 19263, 119021, 100451,  14965])
tensor([ 68204,  99577,  82544, 120521])
tensor([130746,  94992, 100680,  62990])
tensor([18150, 35537, 30789, 45920])
tensor([144081, 121988,  60976,  68852])
tensor([ 97075,  98636, 135458,  72321])
tensor([ 65057, 117736,  39741, 114672])
tensor([ 73956,  75552, 157002,  20380])
tensor([ 23979,  36883, 112714, 134599])
tensor([31726, 53269, 41537, 94737])
tensor([38150, 30047, 94628, 32433])
tensor([131449,  28143,  12017,  35900])
tensor([151355, 145207,  91257, 131870])
tensor([  8084,  65529,  33546, 113408])
tensor([122426,  15218,  16190,  92462])
tensor([   675, 156385, 112979,  32880])
tensor([21962, 45145, 87657,

In [11]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
eval_iters= 200
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

tensor([ 69615, 125058,  23915,   1259])
tensor([ 62591,  34459, 112393,  88475])
tensor([112550,   1224, 125318, 113735])
tensor([ 94278, 103785,  90637, 153469])
tensor([126381,  22819,  39475,  12352])
tensor([ 17376, 154345, 113198, 131942])
tensor([ 68211,  25031, 142543,   3545])
tensor([70283,  2708, 18121, 86406])
tensor([102556, 103123, 106523,  22529])
tensor([ 96620,   1010,  10298, 135931])
tensor([ 77403, 101917, 132540,  73981])
tensor([95113, 18323, 38653, 66484])
tensor([  7051, 126522, 153981,  38978])
tensor([133020, 157071, 113557,  51933])
tensor([126218,  84928, 147585,  72756])
tensor([  7999,  14115,  54464, 113881])
tensor([157483, 158397,  28703,  98747])
tensor([108127,  74153,  13535, 140633])
tensor([146992,  34805,  71954, 154594])
tensor([ 58565, 139690,  85360, 148091])
tensor([ 50253, 124432,   3637,  43188])
tensor([ 56966,  18236, 135070, 124913])
tensor([ 67815, 101637,  13847,  81481])
tensor([121794, 159825, 157211, 114953])
tensor([ 65062, 112556, 

tensor([58813, 45027, 20043, 13344])
tensor([43106, 11885, 43842, 26056])
tensor([15472, 34692,  1140, 10214])
tensor([51698,  6718, 65773, 37343])
tensor([14171, 63385, 15646, 62697])
tensor([41530, 25457,  9903, 65819])
tensor([ 1408,  3656,  1582, 36325])
tensor([24499,  7850, 30734, 49466])
tensor([54188, 45951,  1714, 59463])
tensor([63002,  5707, 11766, 39167])
tensor([ 6709, 26433, 44376,  8170])
tensor([27845, 43572, 21119, 44857])
tensor([ 1261, 19085, 47631, 16675])
tensor([10975, 33412, 29650, 36761])
tensor([53469, 33428, 20258, 19845])
tensor([63265,  6652, 10246, 36174])
tensor([19705, 48175, 44952, 49468])
tensor([30567, 33138, 17400, 18025])
tensor([39348, 65303, 24294,  6939])
tensor([15949, 33851, 12937, 14496])
tensor([31725, 17031, 43579, 32421])
tensor([ 7974, 64829, 44078, 51422])
tensor([63406, 17424, 48446, 50174])
tensor([22056, 10897, 47980, 20066])
tensor([49545, 36544, 18389, 64175])
tensor([55519,   750, 22900, 15950])
tensor([ 6898, 29794, 28259, 37117])
t

tensor([159788, 102712,  54571,  52159])
tensor([139066, 127105, 148160, 105950])
tensor([ 65057,   6522, 155834,  26493])
tensor([58674, 59391, 74652, 90929])
tensor([ 12717, 115796,     17, 132281])
tensor([ 74112, 121987,  45341,  90733])
tensor([102175, 146659,  41427,  49468])
tensor([ 67950, 111787,  93033, 101655])
tensor([138663, 118384, 146037, 129205])
tensor([ 35837, 149941,   7862,  91379])
tensor([ 72829, 123185, 105000,   9175])
tensor([ 61739,  32519, 151940, 148145])
tensor([ 45788,  17028, 107909,  28696])
tensor([ 58171, 148208,  27614,  51341])
tensor([ 82952,  78202,   2275, 144838])
tensor([135828, 140872,  92406,  77402])
tensor([ 55998,  25653, 107600, 130897])
tensor([41650, 70403, 86374, 74159])
tensor([155441, 141878,  60734, 129464])
tensor([110466, 123645,  69046,  71962])
tensor([ 55798,  26421, 151289, 145695])
tensor([ 75298,  97692,   2973, 135438])
tensor([85831, 73588, 84735, 68800])
tensor([ 86934, 113069, 124855,  24442])
tensor([133814, 126775, 1281

tensor([ 45727,  83904, 125556,  44181])
tensor([ 65092, 156211,  59723,  22390])
tensor([ 45208, 144075, 137692,  30823])
tensor([38249, 38851,  2271, 32251])
tensor([ 45511, 117417,  74258, 124053])
tensor([ 57776,  44727,  13218, 119153])
tensor([130634,  40320,  79423, 144850])
tensor([ 74129, 111727, 144745,  12058])
tensor([83343, 79804, 38762, 95614])
tensor([ 90736,  37216,  52703, 116259])
tensor([160133,  64886, 102091, 156504])
tensor([126759,  59487,  73140,  74175])
tensor([ 79381,  34271,  66312, 128288])
tensor([106284, 147119, 127761,  74682])
tensor([ 9721, 38747, 47110, 96864])
tensor([ 51577,  18910, 160884,  52083])
tensor([ 76278,  73530,  35791, 131308])
tensor([128101, 119337,  54737,  70400])
tensor([ 14272,  54940, 159668, 130589])
tensor([ 30050,  90537, 114666,  98858])
tensor([ 15369,  90355, 150600, 152945])
tensor([  8967,  60328, 105240, 133125])
tensor([ 72625,  97208, 129549, 139436])
tensor([ 86064, 118848,  13203,  72186])
tensor([115281, 120672, 1462

tensor([39192,  6556, 55156, 54658])
tensor([44531, 28212, 40435, 35710])
tensor([ 7237, 15737, 30647, 37310])
tensor([ 2754, 36213, 62060, 63389])
tensor([39611, 64416, 29022, 55669])
tensor([17568, 18511, 65317, 63261])
tensor([ 3396,  6026, 13283, 61595])
tensor([ 8748,  9037, 34353,   718])
tensor([24380, 28425, 11767, 23795])
tensor([30688, 24094,  7590,  3949])
tensor([17720, 60449,  3660, 46105])
tensor([49125, 39957, 47919, 10784])
tensor([23557,  3491, 36242, 29521])
tensor([13751, 14898, 11194, 40311])
tensor([24442, 60302, 22970,  6921])
tensor([10986, 29160, 54502, 25311])
tensor([47589, 50424, 32688, 37481])
tensor([66497, 17630, 25119, 14241])
tensor([ 8286, 16634, 17691, 45423])
tensor([16320, 60919, 41476, 47566])
tensor([13621, 33172, 49624, 28621])
tensor([25725, 40609, 32060, 66773])
tensor([38866, 26739, 19525,  2909])
tensor([ 9347, 31408, 31804, 22403])
tensor([14141, 42084, 12438, 63233])
tensor([23562, 31533, 55662, 10556])
tensor([25410, 41751, 48505,  2286])
t

...

tensor([ 23562, 118509,  32076, 155753])
tensor([136547, 160580,  51135,  23885])
tensor([ 10202, 108632,  30095,  93225])
tensor([163160,   9024,  35582, 137317])
tensor([47042, 16777, 62435, 41188])
tensor([ 22266, 113079,  52614, 124238])
tensor([105746,  53444,  77043, 133711])
tensor([127709, 136991,  60173,  18094])
tensor([ 22774, 153248,  60168,  70511])
tensor([153322,  50375,  91896, 134473])
tensor([127855, 108180,   6707,  65841])
tensor([  3944,  49091,  56029, 141701])
tensor([129760,  30657, 126149,  83510])
tensor([126161,  89043,  69307,  59334])
tensor([68383, 63743, 19102, 86225])
tensor([  529, 97915, 93861, 83922])
tensor([146873,  10888, 136060,   4666])
tensor([148641, 132217,  91750, 130747])
tensor([131801,  24732,  76311, 118086])
tensor([ 3033, 11241, 13679, 33493])
tensor([138417, 132595, 143144,  16981])
tensor([ 42663,  97070, 126710, 122226])
tensor([12537, 16562,   819, 13541])
tensor([111723, 125325, 131636,   2240])
tensor([102673,  77599,  20418, 1214

tensor([ 15592, 121823,  19933,  38325])
tensor([111208,  38932,  16211,  93923])
tensor([106020,  28051, 149209,  95285])
tensor([19119, 25535,  8229, 65469])
tensor([29293, 15850, 13703, 59875])
tensor([105499, 106500,  64152,  65609])
tensor([62362, 96223,   177, 74279])
tensor([161513,  18656,  27275, 161112])
tensor([ 51414,  38108, 106038, 139537])
tensor([  2687, 135619, 102529,  47039])
tensor([142354,  21972,  68395, 118378])
tensor([49406, 22500,  6859, 27967])
tensor([ 57003, 116129, 102339, 132394])
tensor([ 92707,  21819,   4263, 105219])
tensor([ 89424, 134007,  30555, 134994])
tensor([ 81488,  74275, 147634,   3373])
tensor([ 63287,  72578,   8600, 157072])
tensor([118666, 133640,  53070,  73471])
tensor([ 55176, 106546,  44091,  21145])
tensor([ 36990, 101445,  80651, 158681])
tensor([35773, 23279, 51612, 15838])
tensor([154510,  92548, 156903, 124461])
tensor([22587, 55298, 64623,  8722])
tensor([132581, 149507, 119975,  36349])
tensor([121860,  77910,  66754,  84368])

tensor([65176, 11125, 25682, 61242])
tensor([12992, 20616, 26383,   390])
tensor([41479, 12886, 12134, 56402])
tensor([61703, 14839, 57844, 35615])
tensor([29553, 12056, 24536, 66476])
tensor([ 9866, 23737, 10907, 35393])
tensor([41887,  1992, 21948, 25668])
tensor([19656, 35867, 45161, 45439])
tensor([  693,  2126, 17251, 56003])
tensor([44282, 57056,  3756, 44383])
tensor([55040, 64536,  9857, 43359])
tensor([27961, 17196, 22640, 46484])
tensor([24710, 46602, 42758, 50102])
tensor([66908, 62813, 18079, 36322])
tensor([63780, 37011, 54008, 54448])
tensor([16880,  6872, 47798, 18864])
tensor([44312, 12557, 18683, 63536])
tensor([50787,  2233, 25925, 29416])
tensor([31349, 61209,  5274, 44489])
tensor([15713, 34983, 21515, 49541])
tensor([42364, 54086,  7454, 13139])
tensor([43478, 30204, 32141, 18377])
tensor([56601, 18526,  7701, 42546])
tensor([43278, 51262, 63150, 39020])
tensor([27648, 47687, 34235, 10609])
tensor([25640, 16990, 66251, 22932])
tensor([36597, 48573, 50015, 64633])
t

tensor([ 13228, 160926,   9048,  40995])
tensor([125787,  52727, 131450, 159972])
tensor([131128, 103623,  16695, 121169])
tensor([ 94647,  48390, 144377,  77815])
tensor([141370,  81710,  95891, 122047])
tensor([ 50063, 152529, 158450,  10071])
tensor([37298, 45320, 41046, 49367])
tensor([  8914, 135756, 139345, 143612])
tensor([101094,  26911, 145571,  22731])
tensor([12364, 86558, 52851, 50119])
tensor([ 72010, 143956,    651, 147568])
tensor([162290, 100759, 146692, 110155])
tensor([110961,  72161,  62680,  90896])
tensor([ 12220, 142903, 128489,  69251])
tensor([164548, 106780,   7588,  57120])
tensor([143517,  89347,  97571,  39255])
tensor([ 42510,  58464, 151622, 108466])
tensor([136222,  85041, 107667,  57005])
tensor([ 26309, 144213, 118846,  74025])
tensor([ 50824,  85457, 116144, 143019])
tensor([147489,  58042, 103979, 113050])
tensor([ 65080,  64593, 151783,   2351])
tensor([ 97364, 139408, 102123, 160261])
tensor([127847, 109932, 132932, 102215])
tensor([67169, 70057, 64

tensor([35697, 33914, 51776, 29528])
tensor([60821, 35695, 47397, 19292])
tensor([11294, 64368, 40236,  2227])
tensor([28072,  4686, 58537, 60153])
tensor([34480, 62280,  2792,  5891])
tensor([17922, 64270, 52446, 45867])
tensor([21930, 33389, 63692, 31194])
tensor([11649, 21197, 48339,  4566])
tensor([48399, 23986, 41000, 30443])
tensor([58925, 23696, 38054, 24686])
tensor([ 6551, 20042, 61607, 40090])
tensor([ 6096,  1818, 19948, 49512])
tensor([38606, 26624,  8403, 59239])
tensor([62698, 54885, 36240, 36507])
tensor([ 8441, 63553, 17325, 28459])
tensor([31143, 22846,  6091, 40397])
tensor([11173, 45922, 14759, 47773])
tensor([10198, 32581,  7836, 34031])
tensor([44006, 40944, 40085, 15961])
tensor([30630, 65564,  3294, 43685])
tensor([60747, 14346,  3772,  6558])
tensor([ 2334, 59842, 26266, 33813])
tensor([47866, 52753, 29727, 55911])
tensor([ 2686, 61821, 38577,  8336])
tensor([12541,  6079, 11573, 40944])
tensor([ 4872, 14164,  2665, 40471])
tensor([57520,  3338,  1948, 57901])
t

In [14]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


"
